## Testing new pipeline features

In [6]:
import sys
import os

# Adds the project root to the path so it can find the 'src' contents
sys.path.append(os.path.abspath(".."))

In [7]:
from src.configs.config import Config
settings = Config.load_ingestion_config()
print(f"RA.CO Enabled: {settings['sources']['ra_co']['enabled']}")

RA.CO Enabled: True


In [8]:
from src.ingestion.pipelines.apis.ra_co import RaCoIngestionPipeline

ImportError: cannot import name 'RaCoIngestionPipeline' from 'src.ingestion.pipelines.apis.ra_co' (/Users/josegarcia/Documents/GitHub/event-intelligence-platform/src/ingestion/pipelines/apis/ra_co.py)